In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm, trange

In [2]:
root = str(Path(os.getcwd()).absolute().parent)
os.chdir(root)

In [3]:
root

'/Users/maximebonnin/Documents/Projects/SCOR/Datathon'

## Try to merge data after concatenate data by year

for year in [2017,2018,2019]:
    pathData = Path(f"RawData/{year}/")
    paths = [pathData / name for name in os.listdir(pathData)]
    dfs = [pd.read_excel(path) for path in paths]
    df_summary = pd.concat(dfs, ignore_index=True)
    df_summary["Year"] = year
    df_summary.to_csv(pathData / f"{year}_summary.csv", index=False)

df_test = pd.read_csv(pathData / f"{year}_summary.csv")
df_test

In [145]:
def add_key_str(df):

    df["GP"] = df["GP"].fillna("")
    df["Block"] = df["Block"].fillna("").astype(str)
    

    df["key_str"] = df["State"].astype(str) + "_" + df["District"].astype(str)
    df["key_str"] += "_" + df["Sub-District"].astype(str) + "_"
    df["key_str"] += df["Block"].astype(str) + "_"
    df["key_str"] += df["GP"].astype(str)

    df["key_str"] = df["key_str"].str.lower()

columns_to_keep = ['Season',
       'Crop', 'Area Sown (Ha)', 'Area Insured (Ha)', 'SI Per Ha (Inr/Ha)',
       'Sum Insured (Inr)', 'Indemnity Level', 'key_str', 'Loss']

yields = ['State', 'Cluster', 'District', 'Sub-District', 'Block', 'GP', 'Season',
        '2000 Yield', '2001 Yield', '2002 Yield', '2003 Yield', '2004 Yield', 
        '2005 Yield', '2006 Yield', '2007 Yield', '2008 Yield', '2009 Yield', 
        '2010 Yield', '2011 Yield', '2012 Yield', '2013 Yield', '2014 Yield', 
        '2015 Yield', '2016 Yield', '2017 Yield', '2018 Yield', 'key_str'
]

def merge_year(dfs):
    df_merged = dfs[0].copy()[columns_to_keep]
    df_merged = df_merged.merge(dfs[1][columns_to_keep], on=["Season", "key_str"],suffixes=('_2017', '_2018'))
    df_merged = df_merged.merge(dfs[2][columns_to_keep], on=["Season", "key_str"],suffixes=("_2018", '_2019'))

    return df_merged


def compute_mean_by_crop(df): 
    stats = {}

    df = df.copy()

    df["Crop"] = df["Crop"].str.lower()
    
    for i in trange(df.shape[0]):
        crop = df.iloc[i]["Crop"]
        if crop not in stats:
            stats[crop] = {}
            stats[crop]["N"] = 0 
            stats[crop]["area_sown"] = 0 
            stats[crop]["area_insured"] = 0
            stats[crop]["si_per_ha"] = 0
            stats[crop]["sum_insured"] = 0
            stats[crop]["indemnity_level"] = 0

            for year in range(2000,2020):
                try:
                    stats[crop][f"yield_{year}"] = 0
                except KeyError:
                    continue


        area_sown = df.iloc[i]["Area Sown (Ha)"]
        area_insured = df.iloc[i]["Area Insured (Ha)"]
        si_per_ha = df.iloc[i]["SI Per Ha (Inr/Ha)"]
        sum_insured = df.iloc[i]["Sum Insured (Inr)"]
        indemnity_level = df.iloc[i]["Indemnity Level"]
        
        stats[crop]["area_sown"] += area_sown
        stats[crop]["area_insured"] += area_insured
        stats[crop]["si_per_ha"] += si_per_ha
        stats[crop]["sum_insured"] += sum_insured
        stats[crop]["indemnity_level"] += indemnity_level
        stats[crop]["N"] += 1

        for year in range(2000,2020):
            try:
                stats[crop][f"yield_{year}"] += df.iloc[i][f"{year} Yield"]
            except KeyError:
                continue

    for crop in stats.keys():
        for index in stats[crop].keys():
            if index != "N":
                stats[crop][index] /= stats[crop]["N"]

    return stats


def clean(df, stats):
    # change it to tack into account 
    # pd.isna instead of this method
    df["Crop"] = df["Crop"].str.lower()

    for crop in stats.keys():
        
        df[df["Crop"] == crop]["Area Sown (Ha)"].astype(float).fillna(stats[crop]["area_sown"], inplace=True)
        df[df["Crop"] == crop]["Area Insured (Ha)"].astype(float).fillna(stats[crop]["area_insured"], inplace=True)
        df[df["Crop"] == crop]["SI Per Ha (Inr/Ha)"].astype(float).fillna(stats[crop]["si_per_ha"], inplace=True)
        df[df["Crop"] == crop]["Sum Insured (Inr)"].astype(float).fillna(stats[crop]["sum_insured"], inplace=True)
        df[df["Crop"] == crop]["Indemnity Level"].astype(float).fillna(stats[crop]["indemnity_level"], inplace=True)
    
    

        for year in range(2000,2019):
            try:
                df[df["Crop"] == crop][f"{year} Yield"].fillna(stats[crop][f"yield_{year}"], inplace=True)
                df[df["Crop"] == crop][f"{year} Yield"].fillna(-1, inplace=True)
            except KeyError:
                continue

    df["Area Sown (Ha)"].fillna(-1, inplace=True)
    df["Area Insured (Ha)"].fillna(-1, inplace=True)
    df["SI Per Ha (Inr/Ha)"].fillna(-1, inplace=True)
    df["Sum Insured (Inr)"].fillna(-1, inplace=True)
    df["Indemnity Level"].fillna(-1, inplace=True)
    
    return df

def clean_area_sown(df):
    newValues = []
    for value in df["Area Sown (Ha)"]:
        try:
            value = float(value)
            newValues.append(value)
        except ValueError:
            newValues.append(np.NaN)
    df["Area Sown (Ha)"] = newValues
    return df
    

def clean_yield(df):
    
    for year in range(2000,2020):
        
        try:
            df[f"{year} Yield"]
        except KeyError:
            continue
        
        newValues = []
        for value in df[f"{year} Yield"]:
            try:
                value = float(value)
                newValues.append(value)
            except ValueError:
                newValues.append(np.NaN)
        df[f"{year} Yield"] = newValues
        # print("Compute Mean...")
        # print("Adding mean...")
        
        
    return df
    
def normalization_other(df):
    df["Area Sown (Ha)"] = (df["Area Sown (Ha)"] - df["Area Sown (Ha)"].mean())/df["Area Sown (Ha)"].std()
    df["Area Insured (Ha)"] = (df["Area Insured (Ha)"] - df["Area Insured (Ha)"].mean())/df["Area Insured (Ha)"].std()
    df["SI Per Ha (Inr/Ha)"] = (df["SI Per Ha (Inr/Ha)"] - df["SI Per Ha (Inr/Ha)"].mean())/df["SI Per Ha (Inr/Ha)"].std()
    df["Sum Insured (Inr)"] = (df["Sum Insured (Inr)"] - df["Sum Insured (Inr)"].mean())/df["Sum Insured (Inr)"].std()
    df["Indemnity Level"] = (df["Indemnity Level"] - df["Indemnity Level"].mean())/df["Indemnity Level"].std()

    return df

def normalization_yield(df):
    for year in range(2000,2019):
        try:
            df[f"{year} Yield"] = (df[f"{year} Yield"] - df[f"{year} Yield"].mean())/df[f"{year} Yield"].std()
        except KeyError:
            continue

    return df


def add_Loss(df,year):
    """return a new_df with a new collumn Loss"""
    Y=np.array([df[f'{y} Yield'] for y in np.arange(year-6,year+1)])
    theta=np.array(df["Indemnity Level"])
    Y=np.partition(Y,2,axis=0)
    Y=Y[2:,:]
    print(Y.shape)
    threshold=np.mean(Y, axis=0)*theta
    S=np.array(df["Sum Insured (Inr)"])
    L=np.sum(S*np.maximum(0,threshold-Y),axis=0)/threshold
    new_df=df
    new_df["Loss"]=L
    return new_df


In [151]:
df = pd.read_excel("Data/RawData/2018/2018_Chhattisgarh_Rabi.xlsx")

In [148]:
stats = compute_mean_by_crop(df)

  0%|          | 0/622 [00:00<?, ?it/s]

In [149]:
df = clean_area_sown(df)
df = clean_yield(df)

In [150]:
df_new = df.copy()
df_new = clean(df_new, stats)
df_new

/opt/homebrew/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,State,Cluster,District,Sub-District,Block,GP,Season,Crop,Area Sown (Ha),Area Insured (Ha),...,2007 Yield,2008 Yield,2009 Yield,2010 Yield,2011 Yield,2012 Yield,2013 Yield,2014 Yield,2015 Yield,2016 Yield
0,Chhattisgarh,1,Balod,Balod,NaN,NaN,Rabi,castor,-1.0,625.142857,...,NaN,NaN,NaN,342.0,276.0,220.0,130.0,349.0,NaN,NaN
1,Chhattisgarh,1,Balod,Dondi,NaN,NaN,Rabi,castor,-1.0,625.142857,...,NaN,NaN,NaN,210.0,180.0,190.0,267.0,160.0,NaN,NaN
2,Chhattisgarh,1,Balod,Dondilohara,NaN,NaN,Rabi,castor,-1.0,625.142857,...,NaN,NaN,NaN,318.0,310.0,250.0,244.0,340.0,NaN,NaN
3,Chhattisgarh,1,Balod,Dondilohara,NaN,NaN,Rabi,castor,-1.0,625.142857,...,NaN,NaN,NaN,318.0,310.0,401.0,209.0,321.0,NaN,NaN
4,Chhattisgarh,1,Balod,Gurur,NaN,NaN,Rabi,castor,-1.0,625.142857,...,NaN,NaN,NaN,103.0,274.0,114.0,191.0,169.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,Chhattisgarh,3,Surguja,Lakhanpur,NaN,NaN,Rabi,castor,-1.0,377.500000,...,244.8,272.0,259.0,183.0,417.0,1105.0,581.0,790.0,NaN,NaN
618,Chhattisgarh,3,Surguja,Udaipur,NaN,NaN,Rabi,castor,-1.0,377.500000,...,244.8,272.0,259.0,326.0,400.0,460.0,614.0,555.0,NaN,NaN
619,Chhattisgarh,3,Surguja,Lundra,NaN,NaN,Rabi,castor,-1.0,377.500000,...,288.0,410.0,259.0,668.0,897.0,450.0,375.0,477.0,NaN,NaN
620,Chhattisgarh,3,Surguja,Lundra,NaN,NaN,Rabi,castor,-1.0,377.500000,...,288.0,410.0,259.0,668.0,897.0,450.0,545.0,428.0,NaN,NaN


In [131]:
df_new["Area Insured (Ha)"].fillna(0).unique().min()

-1.0

In [72]:
pd.DataFrame(columns=["Crop"]).query("Crop == 'rice'", inplace=True)

In [ ]:
dfs = []

for year in range(2017,2020):
    pathData = Path(f"RawData/{year}/")
    df = pd.read_csv(pathData / f"{year}_summary.csv")
    dfs.append(df)


In [62]:
new_dfs = []
for i, df in enumerate(dfs):
    print(f"Year {2017+i} ...")
    print('Adding key...')
    add_key_str(df)
    # print(df.columns)
    print("Adding area sown...")
    clean_area_sown(df)
    print("Adding yield...")
    clean_yield(df)
    print("Adding Loss...")
    df = add_Loss(df, 2015+i)
    print("Cleaning ...")
    df = clean(df)
    print("Normalizing...")
    df = normalization_other(df)
    new_dfs.append(df)

df_merged = merge_year(new_dfs)
df_merged = pd.get_dummies(df_merged, columns=["Crop", "Crop_2017", "Crop_2018"])


Year 2017 ...
Adding key...
Adding area sown...
Adding yield...


  0%|          | 0/20 [00:00<?, ?it/s]

Adding Loss...
(5, 632543)
Cleaning ...
Normalizing...
Year 2018 ...
Adding key...
Adding area sown...
Adding yield...


  0%|          | 0/20 [00:00<?, ?it/s]

Adding Loss...
(5, 633324)
Cleaning ...
Normalizing...
Year 2019 ...
Adding key...
Adding area sown...
Adding yield...


  0%|          | 0/20 [00:00<?, ?it/s]

Adding Loss...
(5, 690412)
Cleaning ...
Normalizing...


In [24]:
df_2019 = new_dfs[-1].copy()
del dfs, df, new_dfs
df_summary_merged = df_merged.merge(df_2019[yields], on=["Season", "key_str"])
df_summary_merged = normalization_yield(df_summary_merged)
df_summary_merged

In [2]:
df_summary_merged.head()

NameError: name 'df_summary_merged' is not defined

## Merge years for every state and season

In [74]:
pathData = Path('Data/RawData')

In [75]:
def extract_infos(filename):
    state, season = filename.split("_")[1], filename.split("_")[-1].replace(".xlsx","")
    return state, season

def key_state_season(filename):
    filename = str(filename)
    return filename.split("_")[1]+"_"+filename.split("_")[-1].replace(".xlsx","")


def merge_DF(paths):

    dfs = [pd.read_excel(path) for path in paths]
    
    new_dfs = []
    for i, df in enumerate(dfs):
        add_key_str(df)
        # print(df.columns)
        clean_area_sown(df)
        clean_yield(df)
        # df = add_Loss(df, 2015+i)
        df = clean(df)
        df = normalization_other(df)
        new_dfs.append(df)

    df_merged = merge_year(new_dfs)
    return df_merged    

In [76]:
allStatesAndSeason = {key_state_season(x) for x in pathData.glob("*/*.xlsx")}
len(allStatesAndSeason)

30

df["Area Sown (Ha)"] = df["Area Sown (Ha)"].astype(float).fillna(-1)
    df["Area Insured (Ha)"] = df["Area Insured (Ha)"].astype(float).fillna(-1)
    df["SI Per Ha (Inr/Ha)"] = df["SI Per Ha (Inr/Ha)"].astype(float).fillna(-1)
    df["Sum Insured (Inr)"] = df["Sum Insured (Inr)"].astype(float).fillna(-1)
    df["Indemnity Level"] = df["Indemnity Level"].astype(float).fillna(-1)

In [77]:
df = pd.read_excel("/Users/maximebonnin/Documents/Projects/SCOR/Datathon/Data/RawData/2018/2018_Andhra Pradesh_Kharif.xlsx")


In [82]:
stats = compute_mean_by_crop(df)



  0%|          | 0/18735 [00:00<?, ?it/s]

AttributeError: 'str' object has no attribute 'keys'

In [69]:
add_key_str(df)
# print(df.columns)
print("Adding area sown...")
clean_area_sown(df)
print("Adding yield...")
clean_yield(df)
print("Adding Loss...")
df = add_Loss(df, 2015+i)
print("Cleaning ...")
df = clean(df)
print("Normalizing...")
df = normalization_other(df)

array(['Arhar', 'Paddy', 'Bajra', 'Castor', 'Chilli IRR', 'Chilli',
       'Cotton Un-IRR', 'Groundnut Un-IRR', 'Groundnut', 'Jowar', 'Maize',
       'Moong', 'Navane', 'Sugarcane Plant', 'Sugarcane Ratoon',
       'Sunflower', 'Urad', 'Chilli Un-IRR', 'Cotton IRR',
       'Groundnut IRR'], dtype=object)

In [68]:
df.query("Crop == 'Paddy'")["SI Per Ha (Inr/Ha)"].fillna(-1).unique()

array([70000, 80000, 71250, 75000, 62500, 72500, 67500, 65000, 40000])

In [44]:
notFull = []
for key in tqdm(allStatesAndSeason):
    paths = [x for x in pathData.glob(f"*/*_{key}.xlsx")]
    paths.sort()
    if len(paths)==3:
        print(paths)
        df = merge_DF(paths)
        print(df.head())
        df.to_csv(pathData / "Unified" / f"{key}.csv")
    else:
        notFull.append(key)
        continue

  0%|          | 0/30 [00:00<?, ?it/s]

[PosixPath('Data/RawData/2017/2017_Rajasthan_Kharif.xlsx'), PosixPath('Data/RawData/2018/2018_Rajasthan_Kharif.xlsx'), PosixPath('Data/RawData/2019/2019_Rajasthan_Kharif.xlsx')]


  0%|          | 0/20 [00:00<?, ?it/s]

(5, 28670)


  0%|          | 0/20 [00:00<?, ?it/s]

...

(5, 13894)


  0%|          | 0/20 [00:00<?, ?it/s]

(5, 17928)
Empty DataFrame
Columns: [Crop_2017, Area Sown (Ha)_2017, Area Insured (Ha)_2017, SI Per Ha (Inr/Ha)_2017, Sum Insured (Inr)_2017, Indemnity Level_2017, Loss_2017, Crop_2018, Area Sown (Ha)_2018, Area Insured (Ha)_2018, SI Per Ha (Inr/Ha)_2018, Sum Insured (Inr)_2018, Indemnity Level_2018, Loss_2018, Season, Crop, Area Sown (Ha), Area Insured (Ha), SI Per Ha (Inr/Ha), Sum Insured (Inr), Indemnity Level, key_str, Loss]
Index: []

[0 rows x 23 columns]
[PosixPath('Data/RawData/2017/2017_Madhya Pradesh_Rabi.xlsx'), PosixPath('Data/RawData/2018/2018_Madhya Pradesh_Rabi.xlsx'), PosixPath('Data/RawData/2019/2019_Madhya Pradesh_Rabi.xlsx')]


  0%|          | 0/20 [00:00<?, ?it/s]

KeyError: '2015 Yield'

In [34]:
"2018" < "2097"

True

In [43]:
notFull

['Bihar_Rabi']